In [1]:
import numpy as np # support for multi-dimensional arrays and matrices
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import mean_absolute_error

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

In [79]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X_test = pd.read_csv('X_test.csv')

In [ ]:
# new feature, 0 if cold (<300 Kelvin), 1 if warm

def is_warm(features):
    warm = []
    for observation in features['reanalysis_avg_temp_k']:
        if observation < 300:
            warm.append(0)
        else:
            warm.append(1)
    return warm

warmth = is_warm(X)
warmth_test = is_warm(X_test)

X['warmth'] = warmth
X_test['warmth'] = warmth_test

In [55]:
# remove constant columns (std = 0)
remove = []
for col in X.columns:
    if X[col].std() == 0:
        remove.append(col)

X.drop(remove, axis=1, inplace=True)
X_test.drop(remove, axis=1, inplace=True)


print(X.shape, X_test.shape)

(1456, 22) (416, 22)


#### Inspecting what a Boosting model selects as features

In [57]:
Cols = X.columns.values.tolist()
clf = GradientBoostingRegressor(random_state = 8001)

selector = clf.fit(X, y)
importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
X = fs.transform(X)
X_test = fs.transform(X_test)
print(train.shape, test.shape)

(1456, 4) (416, 4)


In [29]:
selectedCols = X.shape[1]
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)
X.columns = sortedCols
X_test.columns = sortedCols

print(sortedCols[0:10])

['reanalysis_relative_humidity_percent', 'ndvi_sw', 'station_max_temp_c', 'reanalysis_sat_precip_amt_mm', 'station_min_temp_c', 'station_diur_temp_rng_c', 'precipitation_amt_mm', 'reanalysis_tdtr_k', 'station_precip_mm', 'reanalysis_dew_point_temp_k']


In [30]:
X = X.replace(np.inf, 999999)
X = X.replace(-np.inf, -999999)
X = X.replace(np.nan, -1)
X_test = X_test.replace(np.inf, 999999)
X_test = X_test.replace(-np.inf, -999999)
X_test = X_test.replace(np.nan, -1)

In [13]:
# Second round of gradient boosting
Cols = X.columns.values.tolist()
clf = GradientBoostingRegressor(random_state=1729)
selector = clf.fit(X, y)

importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
X = fs.transform(X)
X_test = fs.transform(X_test)
print(X.shape, X_test.shape)

selectedCols = X.shape[1]
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]

(1456, 4) (416, 4)


In [14]:
import xgboost as xgb

C:\Users\8050116\AppData\Local\conda\conda\envs\iX2017\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

C:\Users\8050116\AppData\Local\conda\conda\envs\iX2017\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Grid Search 1

In [31]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 100, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [32]:
optimized_GBM.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [33]:
optimized_GBM.grid_scores_

[mean: -18.54602, std: 10.41631, params: {'max_depth': 3, 'min_child_weight': 1},
 mean: -18.14835, std: 10.37252, params: {'max_depth': 3, 'min_child_weight': 3},
 mean: -18.18819, std: 10.29187, params: {'max_depth': 3, 'min_child_weight': 5},
 mean: -18.97115, std: 10.47489, params: {'max_depth': 5, 'min_child_weight': 1},
 mean: -18.13049, std: 10.40986, params: {'max_depth': 5, 'min_child_weight': 3},
 mean: -18.05701, std: 10.50860, params: {'max_depth': 5, 'min_child_weight': 5},
 mean: -18.97940, std: 10.38510, params: {'max_depth': 7, 'min_child_weight': 1},
 mean: -18.10577, std: 10.11455, params: {'max_depth': 7, 'min_child_weight': 3},
 mean: -18.02679, std: 10.42581, params: {'max_depth': 7, 'min_child_weight': 5}]

### Grid Search 2

In [34]:
cv_params = {'max_depth': [6,7,8,9], 'min_child_weight': [3,5,7]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 100, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [35]:
optimized_GBM.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [6, 7, 8, 9], 'min_child_weight': [3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [42]:
optimized_GBM.grid_scores_

[mean: -18.15316, std: 10.47515, params: {'max_depth': 6, 'min_child_weight': 3},
 mean: -18.13118, std: 10.51072, params: {'max_depth': 6, 'min_child_weight': 5},
 mean: -18.17445, std: 10.39576, params: {'max_depth': 6, 'min_child_weight': 7},
 mean: -18.10577, std: 10.11455, params: {'max_depth': 7, 'min_child_weight': 3},
 mean: -18.02679, std: 10.42581, params: {'max_depth': 7, 'min_child_weight': 5},
 mean: -18.10508, std: 10.34401, params: {'max_depth': 7, 'min_child_weight': 7},
 mean: -18.06181, std: 10.45274, params: {'max_depth': 8, 'min_child_weight': 3},
 mean: -18.06456, std: 10.38716, params: {'max_depth': 8, 'min_child_weight': 5},
 mean: -18.10027, std: 10.34702, params: {'max_depth': 8, 'min_child_weight': 7},
 mean: -18.01442, std: 10.36392, params: {'max_depth': 9, 'min_child_weight': 3},
 mean: -18.03640, std: 10.37319, params: {'max_depth': 9, 'min_child_weight': 5},
 mean: -18.10027, std: 10.34702, params: {'max_depth': 9, 'min_child_weight': 7}]

We pick max_depth: 7 and min_child_weight: 5
Next we vary n_estimators, subsample, and colsample_bytree

### Grid Search 3

In [47]:
cv_params = {'n_estimators': [75,100,200,300], 'subsample': [0.7,0.8,0.9], 'colsample_bytree': [0.7,0.8,0.9]}
ind_params = {'learning_rate': 0.1, 'min_child_weight': 5, 'seed':0, 'max_depth': 7}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1) 

In [48]:
optimized_GBM.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=5, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [75, 100, 200, 300], 'subsample': [0.7, 0.8, 0.9], 'colsample_bytree': [0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [49]:
optimized_GBM.grid_scores_

[mean: -18.17995, std: 10.37352, params: {'colsample_bytree': 0.7, 'n_estimators': 75, 'subsample': 0.7},
 mean: -18.01511, std: 10.28089, params: {'colsample_bytree': 0.7, 'n_estimators': 75, 'subsample': 0.8},
 mean: -18.14148, std: 10.47475, params: {'colsample_bytree': 0.7, 'n_estimators': 75, 'subsample': 0.9},
 mean: -18.14560, std: 10.26469, params: {'colsample_bytree': 0.7, 'n_estimators': 100, 'subsample': 0.7},
 mean: -18.12500, std: 10.43131, params: {'colsample_bytree': 0.7, 'n_estimators': 100, 'subsample': 0.8},
 mean: -18.11332, std: 10.52588, params: {'colsample_bytree': 0.7, 'n_estimators': 100, 'subsample': 0.9},
 mean: -18.08929, std: 10.29067, params: {'colsample_bytree': 0.7, 'n_estimators': 200, 'subsample': 0.7},
 mean: -17.94986, std: 10.32143, params: {'colsample_bytree': 0.7, 'n_estimators': 200, 'subsample': 0.8},
 mean: -17.88874, std: 10.26905, params: {'colsample_bytree': 0.7, 'n_estimators': 200, 'subsample': 0.9},
 mean: -18.01030, std: 10.29780, params:

#### Tuned Model

In [62]:
xgdmat = xgb.DMatrix(X, y)

In [70]:
params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
            'max_depth':7, 'min_child_weight':5, 'n_estimators': 300} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['rmse'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

In [71]:
cv_xgb.tail(5)

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
295,14.498722,2.256304,2.201803,0.041923
296,14.493549,2.258551,2.194867,0.039638
297,14.492242,2.253179,2.182868,0.039195
298,14.488799,2.253541,2.173350,0.037211
299,14.488221,2.254652,2.166835,0.038850


In [72]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'max_depth':7, 'min_child_weight':5} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 300)

### Tuned Model Predictions

In [74]:
testdmat = xgb.DMatrix(X_test)

In [75]:
y_pred = final_gb.predict(testdmat)

In [78]:
for pred in y_pred:
    print(int(round(pred)))

7
7
9
10
9
11
25
17
30
14
20
6
19
46
35
49
17
78
45
66
87
78
66
65
27
62
59
44
24
29
21
16
14
13
26
17
12
14
13
10
8
17
13
20
5
5
4
-1
4
1
3
5
7
2
3
2
4
12
7
12
5
20
44
35
43
53
71
70
64
74
71
52
118
89
67
48
74
100
70
37
23
21
15
-3
13
-6
13
-5
11
18
10
14
11
24
9
9
12
3
0
12
9
6
5
3
4
10
6
6
2
12
23
30
10
17
25
36
16
24
54
91
72
83
57
43
103
86
107
95
68
54
93
34
29
15
12
15
5
12
9
17
15
10
18
17
11
10
9
10
4
0
1
2
2
11
5
0
6
6
5
2
4
7
7
8
11
16
13
19
21
22
26
52
72
46
63
70
79
59
52
107
99
62
55
44
6
17
20
15
28
10
12
0
9
11
24
27
10
12
6
8
6
4
1
9
8
2
5
4
1
3
5
4
3
8
11
16
24
28
20
24
17
26
45
29
67
72
4
64
67
53
94
154
107
76
82
78
44
23
14
13
11
12
12
15
11
10
12
14
10
9
10
6
4
6
1
5
1
4
4
0
4
4
1
9
4
1
3
3
-5
0
25
-5
2
9
50
9
28
11
-1
6
4
5
5
4
21
7
2
18
11
10
17
25
13
19
18
17
13
10
5
8
4
3
3
6
5
2
2
3
2
1
3
2
3
2
1
4
2
3
6
2
2
7
3
6
11
3
8
5
9
6
13
-3
-1
4
6
-1
22
5
3
14
20
16
21
21
29
16
13
15
9
7
9
7
4
4
5
4
3
4
8
2
5
3
1
2
2
1
1
4
3
1
2
2
4
5
3
4
6
4
0
12
8
9
9
12
-1
9
9
4


### Initial Model (cross validation, no tuning grid)

In [58]:
# Create an empty array for prediction
predictedResult = np.zeros(X.shape[0])

# Split dataset into k = 10 consecutive folds
# Each fold is used once as a validation while the k - 1 remaining folds form the training set
kf = KFold(X.shape[0], n_folds=5)

testPred = []

for trainIndex, testIndex in kf:
    trainFold, testFold = X[trainIndex], X[testIndex]
    trainFoldTarget, testFoldTarget = y[trainIndex], y[testIndex]
    
    xgbc = xgb.XGBRegressor(n_estimators = 300, # number of boosted trees
                             learning_rate = 0.1, # step size shrinkage used in update to prevent overfitting
                             max_depth = 7, # maximum depth of a tree
                             subsample = 0.8, # subsample ratio of the training set (Stochastic gradient boosting)
                             colsample_bytree = 0.8,
                           min_child_weight = 5) # subsample features
    
    xgbc.fit(trainFold, trainFoldTarget)
    xgbpred =xgbc.predict(testFold)

    testPred.append(xgbc.predict(X_test))
    predictedResult[testIndex] = xgbpred
    
    # Print the MA
    print(mean_absolute_error(testFoldTarget, xgbpred))

36.5845090288
21.1651226273
19.8509891143
15.7153119197
13.7039760316


In [59]:
print(mean_absolute_error(y, predictedResult))
testPred = np.average(np.array(testPred), axis =0)
#pd.DataFrame({"ID": test_id, "TARGET": testPred}).to_csv('submission.csv',index=False)

21.4144079307


In [60]:
for pred in testPred:
    print(int(round(pred)))

7
7
9
8
10
10
17
17
21
15
17
13
17
39
34
35
18
67
32
56
80
69
62
61
31
54
60
44
30
32
26
14
21
17
21
18
13
16
11
12
9
15
14
18
7
6
4
-2
5
2
3
5
6
2
3
2
7
10
9
13
9
17
40
30
44
42
55
62
53
63
70
46
90
73
58
42
66
80
72
44
26
26
14
5
12
2
13
-3
14
19
11
16
11
22
11
11
13
4
2
8
8
6
5
3
5
8
6
4
7
10
18
27
8
11
22
33
17
21
42
69
61
79
36
29
83
65
84
87
63
53
76
36
37
19
18
17
7
13
14
20
12
13
19
18
12
10
9
12
8
2
2
3
5
10
5
2
6
7
6
2
4
7
8
8
8
16
9
15
22
22
18
46
64
38
56
54
62
51
43
86
87
54
51
55
15
20
22
14
33
10
10
4
13
15
23
23
11
14
7
11
6
7
4
10
6
3
6
4
3
3
6
5
4
8
10
16
21
25
17
17
21
27
37
28
57
62
11
57
59
46
85
127
92
61
66
68
39
20
16
15
13
14
11
17
13
14
16
11
13
12
10
7
6
6
1
5
3
4
5
1
5
5
2
11
7
5
8
9
1
10
20
-3
9
21
44
24
81
29
14
15
10
12
13
11
14
10
7
19
13
11
16
20
11
18
15
16
11
9
5
8
5
4
3
5
4
4
6
4
4
4
5
4
4
6
5
6
4
7
10
7
6
10
9
12
15
15
22
17
28
12
16
4
6
7
9
6
25
11
3
14
19
15
17
20
23
16
9
13
9
8
6
9
4
6
6
4
4
4
6
3
5
4
4
3
5
4
4
7
6
4
6
7
8
9
7
9
14
14
13
21
18
14